In [15]:
import torch
from datasets import load_dataset
from datasets import load_from_disk
import random, math

import sys
sys.path.append("/home/mylady/code/python/DL-pytorch/apps/chapter_pytorch_demo")
import d2lzh_pytorch.torch as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 日志级别
import transformers


transformers.logging.set_verbosity_error()

In [5]:
#定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)
        dataset = load_from_disk('./data/ChnSentiCorp')['%s' % split]
        def f(data):
            return len(data['text']) > 20
        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset['text'][i]
        
        # print('原始数据: ', text)
        # label = self.dataset['label'][i]
        #切分一句话为前半句和后半句
        #sentence1 = text[:20]
        #sentence2 = text[20:]
        
        #切分一句话为前半句和后半句
        s_len = len(text)
        half_len = math.floor(s_len / 2)  # 切分句子为两段
        
        sentence1 = text[:half_len]
        sentence2 = text[half_len: ]
        label = 0

        # 有一半的概率把后半句替换为一句无关的话
        if random.randint(0, 1) == 0:
            j = random.randint(0, len(self.dataset) - 1)
            # sentence2 = self.dataset[j]['text'][20:40]
            sentence2 = self.dataset['text'][j][half_len:]
            label = 1

        return sentence1, sentence2, label

# 加载
dataset = Dataset('train')
print("数据加载完毕..")

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-2c893bcab2dc48fd.arrow


数据加载完毕..


In [6]:
for i in range(5):
    print(dataset[i])
    print("")

('选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般', '，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 0)

('15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜', '送货要批评一下，星期六下的单，星期三才到。 价格贵了点！', 1)

('1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来.', ' 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.', 0)

('今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当', '一下，我想许多入门玩家会着急一下下。。。', 1)

('机器背面似乎被撕了张什么标签，残胶还在。但', '！感觉上有种内容不咋的，包装挺豪华的感觉！卖的不是内容而是包装！', 1)



In [7]:
# 测试拼接的数据
for t_item in range(5):
    sentence1, sentence2, label = dataset[t_item]
    print(len(dataset), sentence1, '\t',sentence2, label)

9552 选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般 	 通还算方便,离地铁不行10分钟内,但你必须知道哪条是正确的路,如果入住,必须要有心理准备,一个字"小"! 1
9552 15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜 	 欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错 0
9552 1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 	  1
9552 今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当 	 网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。 0
9552 机器背面似乎被撕了张什么标签，残胶还在。但 	 就是作者的一些美容方子有些东西我找不到，呵呵！继续看！ 1


In [9]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path='bert-base-chinese',
    force_download=False,
)

print(token)


# 加载预训练模型
pretrained = BertModel.from_pretrained(
    pretrained_model_name_or_path='bert-base-chinese',
    force_download=False
).to(device)

print('模型加载完毕..')

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
模型加载完毕..


In [31]:
def collate_fn(data):
    sents = [i[:2] for i in data]
    labels = [i[2] for i in data]
    
    # print("sents: ", sents)

    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=200,
                                   return_tensors='pt',
                                   return_length=True,
                                   add_special_tokens=True)

    # input_ids: 编码之后的数字
    # attention_mask: 是补零的位置是0,其他位置是1
    # token_type_ids: 第一个句子和特殊符号的位置是0,第二个句子的位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [32]:

loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print(len(loader))
    print(token.decode(input_ids[0]))
    break

input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

1194
[CLS] 虽 然 很 喜 欢 张 小 娴 的 文 字 ， 面 包 树 系 列 也 很 早 就 看 过 了 ， 但 是 还 是 买 了 她 所 有 自 己 喜 欢 的 书 回 来 珍 藏 。 面 包 树 系 列 故 事 讨 厌 女 主 角 的 个 性 也 不 喜 欢 结 尾 为 了 一 个 自 己 所 谓 的 完 美 爱 情 ， 放 弃 了 那 么 多 [SEP] 可 能 会 很 适 合 自 己 的 优 秀 男 人 ， 不 值 但 是 这 样 对 爱 情 的 执 着 也 许 会 打 动 很 多 人 吧 我 确 实 也 为 书 里 伤 感 的 情 节 而 落 过 泪 感 叹 唏 嘘 但 是 这 不 是 我 要 的 每 个 人 追 求 的 爱 情 不 一 样 ， 有 些 感 受 却 是 可 以 一 样 的 吧 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


(torch.Size([8, 200]),
 torch.Size([8, 200]),
 torch.Size([8, 200]),
 tensor([0, 1, 0, 0, 0, 1, 1, 0]))

In [40]:
len(loader)

597

In [33]:
# 不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

In [34]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out

In [35]:
model = Model()

# 模型转移到GPU上
model.to(device)

Model(
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

In [39]:
from transformers import AdamW


# 训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()


model.train()

num_epochs = 5 

timer = d2l.Timer()
timer.stop()

for epoch in range(num_epochs):
    
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=16,
                                         collate_fn=collate_fn,
                                         shuffle=True,
                                         drop_last=True)
    
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)

        loss = criterion(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            # labels = labels  #.cpu()
            out = out.argmax(dim=1)  #.cpu()
            res_acc = (out == labels).sum().item() # .cpu()
            accuracy = res_acc / len(labels.cpu())
            print('epoch: %s, idx: %s, loss: %s, acc: %s, time: %s' % \
                  (epoch, i, loss.item(), accuracy, timer.final_time()))
        
        #if i % 50 == 0:
        #    #print('epoch: %s, idx: %s, time: %s' % (epoch, i, timer.final_time()))
        
        timer.stop()
    pass
                 
    
# 训练结束
print('总计耗时: ', timer.sum())
print('计算耗时时间段: ', timer.interval_consume())

epoch: 0, idx: 0, loss: 0.382328599691391, acc: 0.9375, time:   0.0000
epoch: 0, idx: 100, loss: 0.3338753581047058, acc: 1.0, time:  58.3937
epoch: 0, idx: 200, loss: 0.4323582053184509, acc: 0.9375, time: 117.6528
epoch: 0, idx: 300, loss: 0.401727557182312, acc: 0.875, time: 176.1585
epoch: 0, idx: 400, loss: 0.5155487060546875, acc: 0.8125, time: 235.0740
epoch: 0, idx: 500, loss: 0.39933276176452637, acc: 0.9375, time: 294.1627
epoch: 1, idx: 0, loss: 0.4222555160522461, acc: 0.875, time: 350.9893
epoch: 1, idx: 100, loss: 0.48862069845199585, acc: 0.8125, time: 409.2094
epoch: 1, idx: 200, loss: 0.4204719066619873, acc: 0.875, time: 468.3088
epoch: 1, idx: 300, loss: 0.5073793530464172, acc: 0.75, time: 527.0634
epoch: 1, idx: 400, loss: 0.3615754246711731, acc: 0.9375, time: 585.7825
epoch: 1, idx: 500, loss: 0.33340275287628174, acc: 1.0, time: 644.6685
epoch: 2, idx: 0, loss: 0.5118041038513184, acc: 0.75, time: 702.2481
epoch: 2, idx: 100, loss: 0.3177926242351532, acc: 1.0, 

In [42]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)
    
    print('测试数据长度: ', len(loader_test))
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        
        #if i == 5:
        #    break
            
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
        pred = out.argmax(dim=1)
        
        # 转移到cpu上
        pred = pred.cpu()
        labels = labels.cpu()
        
        correct += (pred == labels).sum().item()
        total += len(labels)
        
        if i // 200 == 0:
            print(i, 'acc: ', correct / total)
            pass

    # print('epoch: %s acc: %s' % ( correct / total))

In [43]:


test()

Filter:   0%|          | 0/1200 [00:00<?, ? examples/s]

测试数据长度:  37
0 acc:  0.96875
1 acc:  0.953125
2 acc:  0.9270833333333334
3 acc:  0.9296875
4 acc:  0.925
5 acc:  0.921875
6 acc:  0.9241071428571429
7 acc:  0.9296875
8 acc:  0.9201388888888888
9 acc:  0.925
10 acc:  0.9147727272727273
11 acc:  0.921875
12 acc:  0.9206730769230769
13 acc:  0.921875
14 acc:  0.9229166666666667
15 acc:  0.923828125
16 acc:  0.9227941176470589
17 acc:  0.9253472222222222
18 acc:  0.9259868421052632
19 acc:  0.9265625
20 acc:  0.9285714285714286
21 acc:  0.9289772727272727
22 acc:  0.9266304347826086
23 acc:  0.9270833333333334
24 acc:  0.9275
25 acc:  0.9290865384615384
26 acc:  0.9293981481481481
27 acc:  0.9308035714285714
28 acc:  0.9310344827586207
29 acc:  0.9302083333333333
30 acc:  0.9324596774193549
31 acc:  0.931640625
32 acc:  0.9299242424242424
33 acc:  0.9292279411764706
34 acc:  0.9303571428571429
35 acc:  0.9296875
36 acc:  0.9307432432432432


TypeError: not enough arguments for format string

In [44]:
# 保存模型

# 保存

model_save_path = 'chinese_infer_mission_2023_4_20_v4.pt'
torch.save(model.state_dict(),  model_save_path)  # 推荐的文件后缀名是pt或pth
